In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
import scipy.stats as st
from scipy.stats import linregress
import re
from pprint import pprint
import country_converter as coco
import datetime
# cc = coco.CountryConverter()
# from geopy.exc import GeocoderTimedOut 
# from geopy.geocoders import Nominatim 
from config import gkey

In [2]:
countries = 'https://api.covid19api.com/countries'

In [3]:
response_countries = requests.get(countries).json()
pprint(response_countries)

[{'Country': 'Greece', 'ISO2': 'GR', 'Slug': 'greece'},
 {'Country': 'Mozambique', 'ISO2': 'MZ', 'Slug': 'mozambique'},
 {'Country': 'Sudan', 'ISO2': 'SD', 'Slug': 'sudan'},
 {'Country': 'Mexico', 'ISO2': 'MX', 'Slug': 'mexico'},
 {'Country': 'Myanmar', 'ISO2': 'MM', 'Slug': 'myanmar'},
 {'Country': 'Niger', 'ISO2': 'NE', 'Slug': 'niger'},
 {'Country': 'Oman', 'ISO2': 'OM', 'Slug': 'oman'},
 {'Country': 'Tonga', 'ISO2': 'TO', 'Slug': 'tonga'},
 {'Country': 'Korea (North)', 'ISO2': 'KP', 'Slug': 'korea-north'},
 {'Country': 'Liechtenstein', 'ISO2': 'LI', 'Slug': 'liechtenstein'},
 {'Country': 'Maldives', 'ISO2': 'MV', 'Slug': 'maldives'},
 {'Country': 'Monaco', 'ISO2': 'MC', 'Slug': 'monaco'},
 {'Country': 'Guadeloupe', 'ISO2': 'GP', 'Slug': 'guadeloupe'},
 {'Country': 'Macedonia, Republic of', 'ISO2': 'MK', 'Slug': 'macedonia'},
 {'Country': 'Saint Kitts and Nevis',
  'ISO2': 'KN',
  'Slug': 'saint-kitts-and-nevis'},
 {'Country': 'Romania', 'ISO2': 'RO', 'Slug': 'romania'},
 {'Country'

In [4]:
len(response_countries)

248

In [5]:
name = []
slug = []
ISO2 = []
for i in range(len(response_countries)):
    name.append(response_countries[i]["Country"])
    slug.append(response_countries[i]["Slug"])
    ISO2.append(response_countries[i]["ISO2"])

countries_slug_df = pd.DataFrame({"Entity": name,
                                 "slug": slug,
                                 "ISO2": ISO2})
countries_slug_df.head()

,Entity,slug,ISO2
0,Greece,greece,GR
1,Mozambique,mozambique,MZ
2,Sudan,sudan,SD
3,Mexico,mexico,MX
4,Myanmar,myanmar,MM


In [6]:
countries_slug_df.sort_values(by = ["Entity"], inplace = True, ignore_index = True)
countries_slug_df.head()

,Entity,slug,ISO2
0,ALA Aland Islands,ala-aland-islands,AX
1,Afghanistan,afghanistan,AF
2,Albania,albania,AL
3,Algeria,algeria,DZ
4,American Samoa,american-samoa,AS


In [ ]:
countries_slug_df.to_csv("countries_raw.csv")

In [7]:
len(countries_slug_df)

248

In [ ]:
countries_slug_df.describe()

In [8]:
entity1= list(set(countries_slug_df["Entity"]))
standard_names = coco.convert(names=entity1, to='name_short')
standardized_names1_df = pd.DataFrame({"Entity" : entity1,
                                      "standard_names": standard_names}).sort_values(by = "Entity").reset_index(drop = True)
standardized_names1_df.head()

Netherlands Antilles not found in regex


,Entity,standard_names
0,ALA Aland Islands,Aland Islands
1,Afghanistan,Afghanistan
2,Albania,Albania
3,Algeria,Algeria
4,American Samoa,American Samoa


In [9]:
std_countries_slug_df = pd.merge(countries_slug_df, standardized_names1_df, how = "left", left_on='Entity', right_on='Entity')
std_countries_slug_df.sort_values(by = "Entity", inplace = True)
std_countries_slug_df.head()

,Entity,slug,ISO2,standard_names
0,ALA Aland Islands,ala-aland-islands,AX,Aland Islands
1,Afghanistan,afghanistan,AF,Afghanistan
2,Albania,albania,AL,Albania
3,Algeria,algeria,DZ,Algeria
4,American Samoa,american-samoa,AS,American Samoa


In [ ]:
std_countries_slug_df.to_csv("countries_slug.csv")

store response_countries in to a df.
Import csv into a df.
parse the country name column to add hyphens replacing spaces.
do a lookup on response_countries df to get the slugs.
create for loop to check all the slugs and make separate api calls.

In [10]:
stringency_df = pd.read_csv("covid-stringency-index.csv")
stringency_df.head()

,Entity,Code,Date,stringency_index
0,Afghanistan,AFG,1/1/2020,0.0
1,Afghanistan,AFG,1/2/2020,0.0
2,Afghanistan,AFG,1/3/2020,0.0
3,Afghanistan,AFG,1/4/2020,0.0
4,Afghanistan,AFG,1/5/2020,0.0


In [ ]:
len(stringency_df)

In [11]:
stringency_df["Entity"].nunique()

181

In [12]:
stringency_df['month'] = pd.DatetimeIndex(stringency_df['Date']).month
stringency_df['year'] = pd.DatetimeIndex(stringency_df['Date']).year
stringency_df.head()

,Entity,Code,Date,stringency_index,month,year
0,Afghanistan,AFG,1/1/2020,0.0,1,2020
1,Afghanistan,AFG,1/2/2020,0.0,1,2020
2,Afghanistan,AFG,1/3/2020,0.0,1,2020
3,Afghanistan,AFG,1/4/2020,0.0,1,2020
4,Afghanistan,AFG,1/5/2020,0.0,1,2020


In [13]:
stringency_df.loc[stringency_df['Entity'] == "Timor", 'Entity'] = "Timor-leste"

In [ ]:
stringency_df.describe()

In [ ]:
stringency_df.loc[stringency_df['Entity'] == "Timor-leste"]

In [14]:
entity2= list(set(stringency_df["Entity"]))
standard_names2 = coco.convert(names=entity2, to='name_short')
standardized_names2_df = pd.DataFrame({"Entity" : entity2,
                                      "standard_names": standard_names2}).sort_values(by = "Entity").reset_index(drop = True)
standardized_names2_df.head(10)

,Entity,standard_names
0,Afghanistan,Afghanistan
1,Albania,Albania
2,Algeria,Algeria
3,Andorra,Andorra
4,Angola,Angola
5,Argentina,Argentina
6,Aruba,Aruba
7,Australia,Australia
8,Austria,Austria
9,Azerbaijan,Azerbaijan


In [15]:
cleaned_stringency_df = stringency_df.groupby(by = ["Entity", "Code", "month", "year"], as_index = False)["stringency_index"].mean()
cleaned_stringency_df.head(-10)

,Entity,Code,month,year,stringency_index
0,Afghanistan,AFG,1,2020,0.000000
1,Afghanistan,AFG,2,2020,2.010690
2,Afghanistan,AFG,3,2020,40.831935
3,Afghanistan,AFG,4,2020,80.740000
4,Afghanistan,AFG,5,2020,84.260000
...,...,...,...,...,...
2132,Zambia,ZMB,10,2020,45.818387
2133,Zambia,ZMB,11,2020,45.370000
2134,Zambia,ZMB,12,2020,45.370000
2135,Zimbabwe,ZWE,1,2020,1.254194


In [16]:
std_stringency_df = pd.merge(cleaned_stringency_df, standardized_names2_df, how = "left", left_on='Entity', right_on='Entity')
std_stringency_df.sort_values(by = "Entity", inplace = True)
std_stringency_df.head()

,Entity,Code,month,year,stringency_index,standard_names
0,Afghanistan,AFG,1,2020,0.000000,Afghanistan
10,Afghanistan,AFG,11,2020,7.591333,Afghanistan
9,Afghanistan,AFG,10,2020,16.460323,Afghanistan
7,Afghanistan,AFG,8,2020,55.408710,Afghanistan
6,Afghanistan,AFG,7,2020,78.700000,Afghanistan


In [ ]:
std_stringency_df.to_csv("cleaned_stringency.csv")

In [17]:
merged_stringency_df =pd.merge(std_countries_slug_df, std_stringency_df, how = "inner", on = "standard_names")
merged_stringency_df = merged_stringency_df[merged_stringency_df["standard_names"] != "not found"]
merged_stringency_df.rename(columns = {"Entity_x": "Entity", "Code" : "ISO3"}, inplace=True)
merged_stringency_df = merged_stringency_df[["Entity" , "standard_names", "slug", "ISO2", "ISO3", "month", "year", "stringency_index"]]
merged_stringency_df.sort_values( by = ["standard_names", "year", "month"], inplace = True, ignore_index=True)
merged_stringency_df.head()

,Entity,standard_names,slug,ISO2,ISO3,month,year,stringency_index
0,Afghanistan,Afghanistan,afghanistan,AF,AFG,1,2020,0.000000
1,Afghanistan,Afghanistan,afghanistan,AF,AFG,2,2020,2.010690
2,Afghanistan,Afghanistan,afghanistan,AF,AFG,3,2020,40.831935
3,Afghanistan,Afghanistan,afghanistan,AF,AFG,4,2020,80.740000
4,Afghanistan,Afghanistan,afghanistan,AF,AFG,5,2020,84.260000


In [ ]:
merged_stringency_df.to_csv("merged_stringency.csv")

In [ ]:
merged_stringency_df = pd.read_csv("merged_stringency.csv")

In [18]:
merged_stringency_df["standard_names"].nunique()

181

In [ ]:
std_countries_slug_df[["Entity"]][~std_countries_slug_df["standard_names"].isin(set(merged_stringency_df["standard_names"]))].drop_duplicates().reset_index(drop = True)

In [ ]:
std_stringency_df[["Entity"]][~std_stringency_df["standard_names"].isin(list(set(merged_stringency_df["standard_names"])))].drop_duplicates().reset_index(drop = True)

In [ ]:
merged_stringency_df= pd.read_csv("merged_stringency.csv")

In [19]:
country_names= list(set(merged_stringency_df['slug']))
country_names.sort()
country_names

['afghanistan',
 'albania',
 'algeria',
 'andorra',
 'angola',
 'argentina',
 'aruba',
 'australia',
 'austria',
 'azerbaijan',
 'bahrain',
 'bangladesh',
 'barbados',
 'belarus',
 'belgium',
 'belize',
 'benin',
 'bermuda',
 'bhutan',
 'bolivia',
 'bosnia-and-herzegovina',
 'botswana',
 'brazil',
 'brunei',
 'bulgaria',
 'burkina-faso',
 'burundi',
 'cambodia',
 'cameroon',
 'canada',
 'cape-verde',
 'central-african-republic',
 'chad',
 'chile',
 'china',
 'colombia',
 'congo-brazzaville',
 'congo-kinshasa',
 'costa-rica',
 'cote-divoire',
 'croatia',
 'cuba',
 'cyprus',
 'czech-republic',
 'denmark',
 'djibouti',
 'dominica',
 'dominican-republic',
 'ecuador',
 'egypt',
 'el-salvador',
 'eritrea',
 'estonia',
 'ethiopia',
 'faroe-islands',
 'fiji',
 'finland',
 'france',
 'gabon',
 'gambia',
 'georgia',
 'germany',
 'ghana',
 'greece',
 'greenland',
 'guam',
 'guatemala',
 'guinea',
 'guyana',
 'haiti',
 'honduras',
 'hong-kong-sar-china',
 'hungary',
 'iceland',
 'india',
 'indones

In [ ]:
len(country_names)

In [20]:
country_std_names= list(set(merged_stringency_df['standard_names']))
country_std_names.sort()
country_std_names

['Afghanistan',
 'Albania',
 'Algeria',
 'Andorra',
 'Angola',
 'Argentina',
 'Aruba',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bermuda',
 'Bhutan',
 'Bolivia',
 'Bosnia and Herzegovina',
 'Botswana',
 'Brazil',
 'Brunei Darussalam',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cabo Verde',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Central African Republic',
 'Chad',
 'Chile',
 'China',
 'Colombia',
 'Congo Republic',
 'Costa Rica',
 "Cote d'Ivoire",
 'Croatia',
 'Cuba',
 'Cyprus',
 'Czech Republic',
 'DR Congo',
 'Denmark',
 'Djibouti',
 'Dominica',
 'Dominican Republic',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'Eritrea',
 'Estonia',
 'Eswatini',
 'Ethiopia',
 'Faeroe Islands',
 'Fiji',
 'Finland',
 'France',
 'Gabon',
 'Gambia',
 'Georgia',
 'Germany',
 'Ghana',
 'Greece',
 'Greenland',
 'Guam',
 'Guatemala',
 'Guinea',
 'Guyana',
 'Haiti',
 'Honduras',
 'Hong Kong',
 'Hungary',
 'Iceland',
 'India',
 '

In [ ]:
# Target country
# target_country = "India"
lat = []
lng = []
contry_geo_name = []
# Build the endpoint URL
for country in range(len(country_std_names)):
    geo_url = ('https://maps.googleapis.com/maps/api/geocode/json?address={0}&key={1}').format(country_std_names[country], gkey)
    geo_data = requests.get(geo_url).json()
    contry_geo_name.append(geo_data["results"][0]["address_components"][0]["long_name"])
    lat.append(geo_data["results"][0]["geometry"]["location"]["lat"])
    lng.append(geo_data["results"][0]["geometry"]["location"]["lng"])
    time.sleep(1)
# pprint(geo_data)

In [ ]:
contry_coord = pd.DataFrame({"Entity": contry_geo_name,
                            "lat": lat,
                            "lng" : lng,
                            "standard_names" : country_std_names})
contry_coord.head()

In [ ]:
contry_coord.to_csv("contry_coord.csv")

In [21]:
contry_coord = pd.read_csv("contry_coord.csv")

In [22]:
contry_coord["Entity"].nunique()

181

In [ ]:
base_url = 'https://api.covid19api.com/total/country/'
# https://api.covid19api.com/total/country/india/status/confirmed
entity = []
confirmed = []
deaths = []
recovered = []
active = []
date = []

In [ ]:
# test_url = "https://api.covid19api.com/total/country/albania"
# covid_response = requests.get(test_url).json()
# print(len(covid_response))

In [ ]:
for country in range(len(country_names)):
    search_url = base_url + country_names[country]
    covid_response = requests.get(search_url).json()
#     print(len(covid_response))
    for i in range(len(covid_response)):
        entity.append(covid_response[i]["Country"])
        confirmed.append(covid_response[i]["Confirmed"])
        deaths.append(covid_response[i]["Deaths"])
        recovered.append(covid_response[i]["Recovered"])
        active.append(covid_response[i]["Active"])
        date.append(covid_response[i]["Date"])
    time.sleep(1)

In [ ]:
global_covid_df = pd.DataFrame({"Entity" : entity,
                               "Confirmed" : confirmed,
                               "Deaths" : deaths,
                               "Recovered" : recovered,
                               "Active" : active,
                               "Date" : date})
global_covid_df.sort_values(by = ["Entity", "Date"], inplace = True, ignore_index= True)
global_covid_df.head(-10)

In [ ]:
search_url = "https://api.covid19api.com/total/country/india"
covid_response = requests.get(search_url).json()
covid_response[0]["Date"]

In [ ]:
global_covid_df.to_csv("global_covid_stats.csv")

In [23]:
global_covid_df = pd.read_csv("global_covid_stats.csv")

global_covid_df.head()

,Unnamed: 0,Entity,Confirmed,Deaths,Recovered,Active,Date
0,0,Afghanistan,0,0,0,0,2020-01-22T00:00:00Z
1,1,Afghanistan,0,0,0,0,2020-01-23T00:00:00Z
2,2,Afghanistan,0,0,0,0,2020-01-24T00:00:00Z
3,3,Afghanistan,0,0,0,0,2020-01-25T00:00:00Z
4,4,Afghanistan,0,0,0,0,2020-01-26T00:00:00Z


In [24]:
global_covid_df["Confirmed_daily"] =global_covid_df["Confirmed"].diff().fillna(0)
global_covid_df["Deaths_daily"] =global_covid_df["Deaths"].diff().fillna(0)
global_covid_df["Recovered_daily"] =global_covid_df["Recovered"].diff().fillna(0)
global_covid_df["Active_daily"] =global_covid_df["Active"].diff().fillna(0)

# num = global_covid_df._get_numeric_data()
# num[num < 0] = 0
global_covid_df.loc[global_covid_df['Confirmed_daily'] < 0 , 'Confirmed_daily'] = global_covid_df["Confirmed"]

global_covid_df = global_covid_df[['Entity','Date', 'Confirmed_daily', 'Deaths_daily', 'Recovered_daily',
       'Active_daily']]
global_covid_df.head(-10)

,Entity,Date,Confirmed_daily,Deaths_daily,Recovered_daily,Active_daily
0,Afghanistan,2020-01-22T00:00:00Z,0.0,0.0,0.0,0.0
1,Afghanistan,2020-01-23T00:00:00Z,0.0,0.0,0.0,0.0
2,Afghanistan,2020-01-24T00:00:00Z,0.0,0.0,0.0,0.0
3,Afghanistan,2020-01-25T00:00:00Z,0.0,0.0,0.0,0.0
4,Afghanistan,2020-01-26T00:00:00Z,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
57044,Zimbabwe,2020-12-04T00:00:00Z,123.0,1.0,48.0,74.0
57045,Zimbabwe,2020-12-05T00:00:00Z,70.0,10.0,42.0,18.0
57046,Zimbabwe,2020-12-06T00:00:00Z,101.0,0.0,36.0,65.0
57047,Zimbabwe,2020-12-07T00:00:00Z,121.0,3.0,92.0,26.0


In [25]:
global_covid_df["Entity"].nunique()

172

In [ ]:
len(global_covid_df)

In [ ]:
global_covid_df.to_csv("global_covid_daily_test.csv")

In [26]:
global_covid_df['month'] = pd.DatetimeIndex(global_covid_df['Date']).month
global_covid_df['year'] = pd.DatetimeIndex(global_covid_df['Date']).year
global_covid_df.head()

,Entity,Date,Confirmed_daily,Deaths_daily,Recovered_daily,Active_daily,month,year
0,Afghanistan,2020-01-22T00:00:00Z,0.0,0.0,0.0,0.0,1,2020
1,Afghanistan,2020-01-23T00:00:00Z,0.0,0.0,0.0,0.0,1,2020
2,Afghanistan,2020-01-24T00:00:00Z,0.0,0.0,0.0,0.0,1,2020
3,Afghanistan,2020-01-25T00:00:00Z,0.0,0.0,0.0,0.0,1,2020
4,Afghanistan,2020-01-26T00:00:00Z,0.0,0.0,0.0,0.0,1,2020


In [27]:
global_covid_monthly_df = global_covid_df.groupby(["Entity", "month", "year"], as_index = False).agg({"Confirmed_daily": "sum",
                                                                                   "Deaths_daily": "sum",
                                                                                   "Recovered_daily": "sum",
                                                                                   "Active_daily" : "sum"})
global_covid_monthly_df.head(-10)

,Entity,month,year,Confirmed_daily,Deaths_daily,Recovered_daily,Active_daily
0,Afghanistan,1,2020,0.0,0.0,0.0,0.0
1,Afghanistan,2,2020,1.0,0.0,0.0,1.0
2,Afghanistan,3,2020,174.0,4.0,5.0,165.0
3,Afghanistan,4,2020,1952.0,60.0,255.0,1637.0
4,Afghanistan,5,2020,13081.0,194.0,1068.0,11819.0
...,...,...,...,...,...,...,...
2048,Zambia,10,2020,1673.0,17.0,1721.0,-65.0
2049,Zambia,11,2020,1215.0,8.0,1318.0,-111.0
2050,Zambia,12,2020,928.0,16.0,731.0,181.0
2051,Zimbabwe,1,2020,0.0,-373.0,-17729.0,-473.0


In [ ]:
len(global_covid_monthly_df)

In [28]:
std_global_covid_monthly_df = pd.merge(global_covid_monthly_df,standardized_names1_df, how = "left", on = "Entity")
std_global_covid_monthly_df.head(-10)

,Entity,month,year,Confirmed_daily,Deaths_daily,Recovered_daily,Active_daily,standard_names
0,Afghanistan,1,2020,0.0,0.0,0.0,0.0,Afghanistan
1,Afghanistan,2,2020,1.0,0.0,0.0,1.0,Afghanistan
2,Afghanistan,3,2020,174.0,4.0,5.0,165.0,Afghanistan
3,Afghanistan,4,2020,1952.0,60.0,255.0,1637.0,Afghanistan
4,Afghanistan,5,2020,13081.0,194.0,1068.0,11819.0,Afghanistan
...,...,...,...,...,...,...,...,...
2048,Zambia,10,2020,1673.0,17.0,1721.0,-65.0,Zambia
2049,Zambia,11,2020,1215.0,8.0,1318.0,-111.0,Zambia
2050,Zambia,12,2020,928.0,16.0,731.0,181.0,Zambia
2051,Zimbabwe,1,2020,0.0,-373.0,-17729.0,-473.0,Zimbabwe


In [29]:
std_global_covid_monthly_df["standard_names"].nunique()

172

In [30]:
std_global_covid_monthly_stringency_df = pd.merge(std_global_covid_monthly_df, merged_stringency_df , 
                                                  how = "left", on = ["standard_names", "month", "year"])
std_global_covid_monthly_stringency_df = std_global_covid_monthly_stringency_df[['standard_names', 'month', 'year',
        'Confirmed_daily', 'Deaths_daily', 'Recovered_daily', 'Active_daily', 'stringency_index']]
std_global_covid_monthly_stringency_df.head(-10)

,standard_names,month,year,Confirmed_daily,Deaths_daily,Recovered_daily,Active_daily,stringency_index
0,Afghanistan,1,2020,0.0,0.0,0.0,0.0,0.000000
1,Afghanistan,2,2020,1.0,0.0,0.0,1.0,2.010690
2,Afghanistan,3,2020,174.0,4.0,5.0,165.0,40.831935
3,Afghanistan,4,2020,1952.0,60.0,255.0,1637.0,80.740000
4,Afghanistan,5,2020,13081.0,194.0,1068.0,11819.0,84.260000
...,...,...,...,...,...,...,...,...
2048,Zambia,10,2020,1673.0,17.0,1721.0,-65.0,45.818387
2049,Zambia,11,2020,1215.0,8.0,1318.0,-111.0,45.370000
2050,Zambia,12,2020,928.0,16.0,731.0,181.0,45.370000
2051,Zimbabwe,1,2020,0.0,-373.0,-17729.0,-473.0,1.254194


In [31]:
std_global_covid_monthly_stringency_df["standard_names"].nunique()

172

In [32]:
main_df = pd.merge(std_global_covid_monthly_stringency_df, contry_coord, how = "left", on = "standard_names")
main_df = main_df[['standard_names', 'month', 'year', 'Confirmed_daily', 'Deaths_daily',
       'Recovered_daily', 'Active_daily', 'stringency_index', 'lat','lng']]
main_df.rename(columns = {'Confirmed_daily' : "Confirmed", 'Deaths_daily' : "Deaths",
                           'Recovered_daily': "Recovered", 'Active_daily' : "Active"}, inplace = True)
main_df.head(-10)

,standard_names,month,year,Confirmed,Deaths,Recovered,Active,stringency_index,lat,lng
0,Afghanistan,1,2020,0.0,0.0,0.0,0.0,0.000000,33.939110,67.709953
1,Afghanistan,2,2020,1.0,0.0,0.0,1.0,2.010690,33.939110,67.709953
2,Afghanistan,3,2020,174.0,4.0,5.0,165.0,40.831935,33.939110,67.709953
3,Afghanistan,4,2020,1952.0,60.0,255.0,1637.0,80.740000,33.939110,67.709953
4,Afghanistan,5,2020,13081.0,194.0,1068.0,11819.0,84.260000,33.939110,67.709953
...,...,...,...,...,...,...,...,...,...,...
2048,Zambia,10,2020,1673.0,17.0,1721.0,-65.0,45.818387,-13.133897,27.849332
2049,Zambia,11,2020,1215.0,8.0,1318.0,-111.0,45.370000,-13.133897,27.849332
2050,Zambia,12,2020,928.0,16.0,731.0,181.0,45.370000,-13.133897,27.849332
2051,Zimbabwe,1,2020,0.0,-373.0,-17729.0,-473.0,1.254194,-19.015438,29.154857


In [33]:
main_df["standard_names"].nunique()

172

In [ ]:
main_df.to_csv("main.csv")

In [ ]:
main_stats_df = main_df.groupby("standard_names", as_index = False).agg({"Confirmed": "sum",
                                                                                   "Deaths": "sum",
                                                                                   "Recovered": "sum",
                                                                                   "Active" : "sum"})
main_stats_df.sort_values('Confirmed', inplace= True, ascending=False, ignore_index=True)

main_stats_df.head(10)

In [ ]:
main_stats_df.sort_values('Confirmed', ascending=True, ignore_index=True)

In [ ]:
main_stats_df[:20].to_csv("main_stats_2020.csv")

In [34]:
popluation_2020 = pd.read_csv("population_by_country_2020.csv")
popluation_2020.head(-10)

,Country (or dependency),Population (2020),Yearly Change,Net Change,Density (P/Km²),Land Area (Km²),Migrants (net),Fert. Rate,Med. Age,Urban Pop %,World Share
0,China,1440297825,0.39 %,5540090,153,9388211,-348399.0,1.7,38,61 %,18.47 %
1,India,1382345085,0.99 %,13586631,464,2973190,-532687.0,2.2,28,35 %,17.70 %
2,United States,331341050,0.59 %,1937734,36,9147420,954806.0,1.8,38,83 %,4.25 %
3,Indonesia,274021604,1.07 %,2898047,151,1811570,-98955.0,2.3,30,56 %,3.51 %
4,Pakistan,221612785,2.00 %,4327022,287,770880,-233379.0,3.6,23,35 %,2.83 %
...,...,...,...,...,...,...,...,...,...,...,...
220,Caribbean Netherlands,26265,0.94 %,244,80,328,NaN,N.A.,N.A.,75 %,0.00 %
221,Palau,18109,0.48 %,86,39,460,NaN,N.A.,N.A.,N.A.,0.00 %
222,Cook Islands,17567,0.09 %,16,73,240,NaN,N.A.,N.A.,75 %,0.00 %
223,Anguilla,15026,0.90 %,134,167,90,NaN,N.A.,N.A.,N.A.,0.00 %


## Get 10 countries around mean, 10 countries with least numbers also

In [35]:
popluation_2020["Country (or dependency)"].nunique()

235

In [39]:
popluation_2020[["Country (or dependency)"]][~popluation_2020["Country (or dependency)"].isin(list(set(main_df["standard_names"])))].drop_duplicates().reset_index(drop = True)

,Country (or dependency)
0,Côte d'Ivoire
1,North Korea
2,Czech Republic (Czechia)
3,Hong Kong
4,Kyrgyzstan
...,...
66,Montserrat
67,Falkland Islands
68,Niue
69,Tokelau


In [37]:
entity3= list(set(popluation_2020["Country (or dependency)"]))
standard_names3 = coco.convert(names=entity3, to='name_short')
standardized_names3_df = pd.DataFrame({"Country (or dependency)" : entity3,
                                      "standard_names": standard_names3}).sort_values(by = "Country (or dependency)").reset_index(drop = True)
standardized_names3_df.head(10)

Channel Islands not found in regex


,Country (or dependency),standard_names
0,Afghanistan,Afghanistan
1,Albania,Albania
2,Algeria,Algeria
3,American Samoa,American Samoa
4,Andorra,Andorra
5,Angola,Angola
6,Anguilla,Anguilla
7,Antigua and Barbuda,Antigua and Barbuda
8,Argentina,Argentina
9,Armenia,Armenia


In [40]:
std_population_df = pd.merge(popluation_2020, standardized_names3_df, how = "left", on = "Country (or dependency)")
std_population_df = std_population_df[["standard_names", "Population (2020)"]].dropna()
std_population_df.head()

,standard_names,Population (2020)
0,China,1440297825
1,India,1382345085
2,United States,331341050
3,Indonesia,274021604
4,Pakistan,221612785


In [41]:
std_population_df["standard_names"].nunique()

235

In [42]:
ultimate_df = pd.merge(main_df, std_population_df, how = "inner", on = "standard_names")
ultimate_df["Infection_rate"] = ((ultimate_df["Confirmed"]/ ultimate_df["Population (2020)"]*100).map("{:.4f}%".format))
ultimate_df = ultimate_df[["standard_names", "month", "year", "Confirmed", "stringency_index"
                          , "lat", "lng", "Population (2020)", "Infection_rate" ]]
ultimate_df.head()

,standard_names,month,year,Confirmed,stringency_index,lat,lng,Population (2020),Infection_rate
0,Afghanistan,1,2020,0.0,0.000000,33.93911,67.709953,39074280,0.0000%
1,Afghanistan,2,2020,1.0,2.010690,33.93911,67.709953,39074280,0.0000%
2,Afghanistan,3,2020,174.0,40.831935,33.93911,67.709953,39074280,0.0004%
3,Afghanistan,4,2020,1952.0,80.740000,33.93911,67.709953,39074280,0.0050%
4,Afghanistan,5,2020,13081.0,84.260000,33.93911,67.709953,39074280,0.0335%


In [43]:
ultimate_df["standard_names"].nunique()

171

In [45]:
main_df[["standard_names"]][~main_df["standard_names"].isin(list(set(ultimate_df["standard_names"])))].drop_duplicates().reset_index(drop = True)

,standard_names
0,Kosovo


In [ ]:
ultimate_df.head(20)

In [ ]:
ultimate_df.to_csv("ultimate.csv")

In [ ]:
ultimate_df = pd.read_csv("ultimate.csv")

In [46]:
ultimate_nan_df = ultimate_df[ultimate_df["stringency_index"].isnull()]
ultimate_nan_df

,standard_names,month,year,Confirmed,stringency_index,lat,lng,Population (2020),Infection_rate
11,Afghanistan,12,2020,3347.0,NaN,33.939110,67.709953,39074280,0.0086%
83,Australia,12,2020,214.0,NaN,-25.274398,133.775136,25550683,0.0008%
95,Austria,12,2020,52457.0,NaN,47.516231,14.550072,9015361,0.5819%
515,Denmark,12,2020,47872.0,NaN,56.263920,9.501785,5795780,0.8260%
563,Ecuador,12,2020,12318.0,NaN,-1.831239,-78.183406,17688599,0.0696%
827,India,12,2020,541790.0,NaN,20.593684,78.962880,1382345085,0.0392%
1043,Lesotho,12,2020,437.0,NaN,-29.609988,28.233608,2145194,0.0204%
1138,Malaysia,12,2020,25119.0,NaN,4.210484,101.975766,32436963,0.0774%
1219,Monaco,9,2020,218.0,NaN,43.738418,7.424616,39290,0.5548%
1220,Monaco,10,2020,141.0,NaN,43.738418,7.424616,39290,0.3589%


In [48]:
ultimate_nan_df.to_csv("ultimate_nan.csv", index=False)

In [50]:
ultimate_no_null_df = ultimate_df.dropna()
ultimate_no_null_df.head(20)

,standard_names,month,year,Confirmed,stringency_index,lat,lng,Population (2020),Infection_rate
0,Afghanistan,1,2020,0.0,0.000000,33.939110,67.709953,39074280,0.0000%
1,Afghanistan,2,2020,1.0,2.010690,33.939110,67.709953,39074280,0.0000%
2,Afghanistan,3,2020,174.0,40.831935,33.939110,67.709953,39074280,0.0004%
3,Afghanistan,4,2020,1952.0,80.740000,33.939110,67.709953,39074280,0.0050%
4,Afghanistan,5,2020,13081.0,84.260000,33.939110,67.709953,39074280,0.0335%
5,Afghanistan,6,2020,16299.0,80.182667,33.939110,67.709953,39074280,0.0417%
6,Afghanistan,7,2020,5158.0,78.700000,33.939110,67.709953,39074280,0.0132%
7,Afghanistan,8,2020,1494.0,55.408710,33.939110,67.709953,39074280,0.0038%
8,Afghanistan,9,2020,1109.0,23.520000,33.939110,67.709953,39074280,0.0028%
9,Afghanistan,10,2020,2157.0,16.460323,33.939110,67.709953,39074280,0.0055%


In [51]:
ultimate_no_null_df.to_csv("ultimate_no_null.csv" , index=False)

In [ ]:
ultimate_df["standard_names"].nunique()

In [ ]:
main_df = pd.read_csv("main.csv")

In [ ]:
main_df["standard_names"].nunique()